In [2]:
!git clone https://github.com/KuangDD/zhrtvc.git

Cloning into 'zhrtvc'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 192 (delta 20), reused 61 (delta 13), pack-reused 105
Receiving objects: 100% (192/192), 145.40 MiB | 11.46 MiB/s, done.
Resolving deltas: 100% (23/23), done.
Checking out files: 100% (94/94), done.


In [16]:
!pwd
import os
os.chdir("/content/zhrtvc/zhrtvc");
!pwd

/content
/content/zhrtvc/zhrtvc


In [18]:
# Install dependencies
!pwd
!pip install -q -r /content/zhrtvc/zhrtvc/requirements.txt
!apt-get install -qq libportaudio2

/content/zhrtvc/zhrtvc
     |████████████████████████████████| 377.0 MB 20 kB/s 
     |████████████████████████████████| 66 kB 4.4 MB/s 
     |████████████████████████████████| 63.5 MB 20 kB/s 
     |████████████████████████████████| 488 kB 54.8 MB/s 
     |████████████████████████████████| 3.1 MB 52.4 MB/s 
     |████████████████████████████████| 252 kB 64.3 MB/s 
ERROR: tensorflow 1.15.2 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow 1.15.2 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.14.0 which is incompatible.
ERROR: tensorflow 1.15.2 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.14.0 which is incompatible.


In [0]:
import sys
sys.path.append('/content/zhrtvc/zhrtvc/phkit-0.0.4/phkit')
sys.path.append('/content/zhrtvc/zhrtvc/synthesizer')

In [0]:
# Code for recording audio from the browser
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import IPython
import uuid
from google.colab import output


class InvokeButton(object):
  def __init__(self, title, callback):
    self._title = title
    self._callback = callback

  def _repr_html_(self):
    from google.colab import output
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)

    template = """<button id="{callback_id}" style="cursor:pointer;background-color:#EEEEEE;border-color:#E0E0E0;padding:5px 15px;font-size:14px">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id)
    return html

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('audio.wav','wb+') as f:
    f.write(b)
  return 'audio.wav'

In [26]:
from IPython.display import Audio
from IPython.utils import io
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path
import numpy as np
import librosa
encoder_weights = Path("/content/zhrtvc/pretrained/encoder/saved_models/pretrained.pt")
vocoder_weights = Path("/content/zhrtvc/pretrained/synthesizer/saved_models/logs-sync")
syn_dir = Path("/content/zhrtvc/pretrained/synthesizer/saved_models/logs-sync")
encoder.load_model(encoder_weights)
synthesizer = Synthesizer(syn_dir)
vocoder.load_model(vocoder_weights)

ModuleNotFoundError: ignored